# Discussion 09 Notebook

This notebook is an accompaniment to the associated discussion worksheet handout.

# Section II: Entity Resolution

## Database Setup

In [ ]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS disc07'
!psql -h localhost -c 'CREATE DATABASE disc07'

%reload_ext sql
%sql postgresql://127.0.0.1:5432/disc07

In [ ]:
!psql postgresql://127.0.0.1:5432/disc07 <disc07.sql

## Initial Exploration

Let us first take a look at the content of the database.

In [ ]:
%sql SELECT * FROM albums;

In [ ]:
%sql SELECT * FROM sales;

There is one significant barrier to performing data analysis: joins. Although we have cleaned data locally, within each table, we have not cleaned up the connections between these tables such as the strings we may want to join on.

In our dataset, we'll notice that simply joining on equality of names will result in missing data.

### 1. Let's try it out: write a query to join the tables using strict equality

In [ ]:
%%sql
SELECT a.name, a.artist, b.day, b.count
FROM albums AS a INNER JOIN sales AS b
ON a.name = b.name

We see that we are missing a lot of data, which could cause trouble in computations later on.

## Distance Functions on Strings
The Levenshtein distance function can be helpful to find strings that are _similar_ but not identical. This algorithm computes the minimal number of insertions, deletions, and mutations. We can apply this to our dataset.

### 2. First, let's compute the Levenshtein distance between every pair of names in the two tables, sorted by this distance.

In [ ]:
%%sql
SELECT a.name, b.name, levenshtein(a.name, b.name) AS distance
FROM albums AS a, sales AS b
ORDER BY levenshtein(a.name, b.name) ASC

You'll notice that it's a very close boundary between the strings we want to match and those we don't. In reality, a clustering-based approach would be better suited for this scenario, but let's keep going with distance since that's a lot easier to implement.

We can use `< 10` as our threshold for matching strings.

### 3. Create a materialized view `joined_sales` that joins the tables using Levenshtein distance

Make sure to have the following columns in the view:
- `name`: name of the album
- `artist`: name of the artist
- `day`: day of sales
- `count`: number of sale

In [ ]:
%%sql
DROP MATERIALIZED VIEW IF EXISTS joined_sales;
CREATE MATERIALIZED VIEW joined_sales AS
SELECT a.name, a.artist, b.day, b.count
FROM albums AS a INNER JOIN sales AS b
ON levenshtein(a.name, b.name) < 10;

SELECT * FROM joined_sales;

# Section III: Data Granularity

## Initial Exploration

In [ ]:
%sql SELECT * FROM nodes ORDER BY tax_id LIMIT 5;

In [ ]:
%sql SELECT * FROM names ORDER BY tax_id LIMIT 5;

### 1. Write a SQL query to find the node representing the Animalia kingdom.

In [ ]:
%%sql
SELECT name_txt, nodes.* 
  FROM nodes NATURAL JOIN names
 WHERE name_txt = 'Animalia';

### 2. Let us drill down into the Animalia kingdom. First, find all children nodes of the Animalia kingdom.

In [ ]:
%%sql
SELECT parent.tax_id, parent.rank, child.tax_id, child.rank
  FROM nodes parent, nodes child
 WHERE child.parent = parent.tax_id
   AND parent.tax_id = 33208;

### 3. Next, find the names of these nodes, along with the names of their parents.

In [ ]:
%%sql
WITH edges AS (
    SELECT parent.tax_id AS pid, parent.rank as prank,
           child.tax_id AS cid, child.rank as crank
      FROM nodes parent, nodes child
     WHERE child.parent = parent.tax_id
)
SELECT pid, prank, parent.name_txt as ptext, cid, crank, child.name_txt as ctext
  FROM names parent, names child, edges
 WHERE parent.tax_id = edges.pid
   AND child.tax_id = edges.cid
   AND parent.tax_id = 33208
 LIMIT 10;

### 4. You will find there are many synonym names for the same phylum with the same tax_id. Aggregate them up by grouping by tax_id.

In [ ]:
%%sql
WITH edges AS (
    SELECT parent.tax_id AS pid, parent.rank as prank,
           child.tax_id AS cid, child.rank as crank
      FROM nodes parent, nodes child
     WHERE child.parent = parent.tax_id
)
SELECT pid, edges.prank, json_agg(parent.name_txt), cid, edges.crank, json_agg(child.name_txt)
  FROM names parent, names child, edges
 WHERE parent.tax_id = edges.pid
   AND child.tax_id = edges.cid
   AND parent.tax_id = 33208
 GROUP BY pid, edges.prank, cid, edges.crank;

### 5. Challenge: How can we drill down one more layer? What if we want to get the names of all the classes under the Animalia kingdom?

In [ ]:
%%sql
WITH double_edges AS (
    SELECT grandparent.tax_id AS gid, grandparent.rank as grank, 
           parent.tax_id AS pid, parent.rank as prank,
           child.tax_id AS cid, child.rank as crank
      FROM nodes grandparent, nodes parent, nodes child
     WHERE child.parent = parent.tax_id
       AND parent.parent = grandparent.tax_id
)
SELECT gid, grank, json_agg(grandparent.name_txt), 
       pid, prank, json_agg(parent.name_txt), 
       cid, crank, json_agg(child.name_txt)
  FROM names grandparent, names parent, names child, double_edges de
 WHERE parent.tax_id = de.pid
   AND child.tax_id = de.cid
   AND grandparent.tax_id = de.gid
   AND grandparent.tax_id = 33208
 GROUP BY gid, grank, pid, prank, cid, crank;